In [1]:
import numpy as np
import bruno

### Gerando valores de anomalia de gravidade mGal

In [2]:
N = 6 # numero de nós 
# valor de referencia de gravidade em mGal
g_ref = 978163.000
print(g_ref)

978163.0


In [3]:
# valores verdaeirs de gravidade em cada nó 
rng = np.random.default_rng(seed=78888880864342)
g_true = g_ref + 200.*rng.random(N)

for gi in g_true:
    print(gi)

# indices das estações gi e gj
# usado para calcular as gravidades diferentes
i = [0, 1, 2, 3, 2]
j = [1, 2, 5, 4, 3]

978177.9033183053
978194.2397607918
978207.5782130521
978247.6715835293
978362.1955659175
978273.5946933671


In [4]:
# verdadeira diferença de gravidade
l_true = g_true[j] - g_true[i]

for li in l_true:
    print(li)

16.336442486499436
13.33845226035919
66.016480315011
114.52398238820024
40.09337047720328


In [5]:
# valores verdadeirs de gravidade absoluta
g_abs_true = g_true[[0, 5]]

for gabs in g_abs_true:
    print(gabs)

978177.9033183053
978273.5946933671


In [6]:
# Variância dos dados de gravidade absoluta em (mGal)**2
var_g_abs = 0.008

# Variância dos dados de gravidade diferencial em (mGal)**2
var_l = 0.030

# dados de diferenças contaminados por ruído
noise_l = rng.normal(loc=0., scale=np.sqrt(var_l), size=l_true.size)
l = l_true + noise_l
l = np.round(l, decimals=3)

for li in l:
    print(li)
l

16.578
13.31
66.119
114.582
40.164


array([ 16.578,  13.31 ,  66.119, 114.582,  40.164])

In [7]:
# dados absolutos contaminados por ruído
noise_abs = rng.normal(loc=0., scale=np.sqrt(var_g_abs), size=g_abs_true.size)
g_abs = g_abs_true + noise_abs
g_abs = np.round(g_abs, decimals=3)
    
for gabsi in g_abs:
    print(gabsi)

978178.137
978273.521


In [8]:
#Create the w using the variances var_g_abs and var_l;
w = np.zeros(7)
for i in range(0, 7):
    if i >=0 and i<5:
        w[i] = var_l
    else:
        w[i] = var_g_abs

w

array([0.03 , 0.03 , 0.03 , 0.03 , 0.03 , 0.008, 0.008])

In [9]:
i = [0, 1, 2, 3, 2]
j = [1, 2, 5, 4, 3]

d = np.hstack([l, g_abs]).T # vetor calculado para L, g absoluto (0 e 5)
d

array([1.65780000e+01, 1.33100000e+01, 6.61190000e+01, 1.14582000e+02,
       4.01640000e+01, 9.78178137e+05, 9.78273521e+05])

In [10]:
# Criando matriz A de acordo com vetor l
A = np.zeros((7, N))
for k in range(5):
    A[k, i[k]] = -1
    A[k, j[k]] = 1
A

array([[-1.,  1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  1.],
       [ 0.,  0.,  0., -1.,  1.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [11]:
g_hat, sigma_g, std = bruno.minimos_quadrados_ponderado(A, d, w, check_input=True, inc = True)

C:\Users\7michelson\Documents\MCOM scripts\Exercícios - Construindo funções - ALgebra linear\bruno.py:1462: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt(np.diag(covariance_matrix))


In [12]:
g_hat

array([-192.,  -64.,  -64., -128.,    0.,  128.])

In [13]:
bruno.minimos_quadrados(A, d)

array([128., 128., 128.,   0., 128.,   0.])

In [14]:
A@g_hat

array([128.,   0., 192., 128., -64.,   0.,   0.])

In [15]:
for s in range(0, len(sigma_g)):
    print(sigma_g[s])

[       0.        0.        0. -2097152. -2097152.        0.]
[       0.        0.        0. -2097152. -2097152.        0.]
[       0.        0.        0. -2097152. -2097152.        0.]
[-2097152. -2097152. -2097152.        0. -2097152. -2097152.]
[-2097152. -2097152. -2097152. -2097152. -1048576. -2097152.]
[       0.        0.        0. -1048576. -1048576.  1048576.]


In [16]:
std

array([   0.,    0.,    0.,    0.,   nan, 1024.])